In [1]:
import pandas as pd
import re
import string
import plotly.express as px
import statistics
import math
from datetime import datetime
import collections
from myFunctions import processSentences, meanVector, cosineDistance
import plotly.express as px


import itertools

# NLP Packages
from gensim.models.phrases import Phrases, Phraser
from gensim import matutils
from gensim.models import KeyedVectors

/Users/brianward/Desktop/Northeastern/AI/project/aiEnv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Now let's use our model to try to extract the company Names

### Goal
* extract company names and tickers using our tool.
* explore what we found
* perform sentiment analysis



In [2]:
# import data
reddit = pd.read_csv("data/reddit_wsb.csv")
reddit2 = pd.read_csv("data/r_wallstreetbets_posts.csv")

/Users/brianward/Desktop/Northeastern/AI/project/aiEnv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Lets pull out our sentences
# Again we are going to take just the titles from one and the blah blah blah

sentences_raw = []


for i in range(len(reddit)):
    body = reddit['body'][i]
    if isinstance(body, str):
        sentences_raw.append(body)
        
for i in range(len(reddit2)):
    title = reddit2['title'][i]
    if isinstance(title, str):
        sentences_raw.append(title)

In [4]:
# process sentences
sentences_processed = processSentences(sentences_raw, minStringSize = 5, minTokenCount = 3, splitonPeriod = False, phraseMinCount = 25)


In [5]:
sentences_processed[2]

['life',
 "isn't",
 'fair',
 'my',
 'mother',
 'always',
 'told_me',
 'that',
 'when',
 'i',
 'would',
 'complain',
 'about',
 'arbitrary',
 'treatment',
 'i',
 'would',
 'play',
 'by',
 'the',
 'rules',
 'and',
 'someone_else',
 'would',
 'ignore',
 'them',
 'when',
 'they',
 'would',
 'win',
 'i',
 'would',
 'appeal',
 'to',
 'the',
 'first',
 'authority',
 'for',
 'an_explanation',
 'are',
 'you',
 'going',
 'to',
 'let_them',
 'get',
 'away',
 'with',
 'this',
 'life',
 "isn't",
 'fair',
 'no',
 'it',
 'is',
 'not',
 'the',
 'game',
 'is',
 'the',
 'game',
 'always',
 'in',
 'this',
 'moment',
 'the',
 'fascade',
 'cracks',
 'further',
 'when',
 'the',
 'first',
 'breach',
 'was',
 'made',
 'i',
 'do',
 'not',
 'know',
 'perhaps',
 'it',
 'was',
 'socrates',
 'but',
 'today',
 'i',
 'see',
 'thousands',
 'millions',
 'once',
 'they',
 'were',
 'laughing',
 'luxuries',
 'falling',
 'out',
 'of',
 'their',
 'disgusting',
 'diseased',
 'mouths',
 'as',
 'they',
 'cackled',
 'the',
 'u

In [6]:
# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("V2.wordvectors", mmap='r')

# Build Extraction Tool

In [7]:



def extractCompanies(keyedVectors, sentences_processed, threshold, myVector):
    
    sentences, extracted = [], []
    
    for sentence in sentences_processed:
        
        found = []
        
        for token in sentence:
        
            try:
                
                if cosineDistance(keyedVectors[token], myVector) >= threshold:
                    found.append(token)
            
            except KeyError:
                pass
        
        if len(found) > 0:
            sentences.append(sentence)
            extracted.append(list(set(found)))
        
    return pd.DataFrame({'sent':sentences, 'extracted':extracted})
            


In [8]:
# calculate our Vec 

myVec = meanVector(wv, positive=['gme', 'amc','game_stop','tsla', 'nok', 'etsy', 'crox', 'appl', 'microsoft', 'facebook', 'netflix', 'dropbox','slack', 'peloton', 'wal_mart', 'lululemon'], 
                negative=[])


In [9]:
cosineDistance(wv['gme'], myVec)

0.7348057

In [17]:
start = datetime.now()
results = extractCompanies(wv, sentences_processed, 0.6, myVec)
end = datetime.now()

print('time elapsed = ', end-start)

time elapsed =  0:04:58.278445


In [18]:
index = 14

print(' '.join(results['sent'][index]))
print(results['extracted'][index])


original article https_www coinspeaker com gamestop blackberry amc trade halts https_www coinspeaker com gamestop blackberry amc trade halts “in the interest of mitigating risk for our company and clients we have put in place several restrictions on some transactions in gme amc and other securities ” a spokeswoman for td_ameritrade told marketwatch referring to the ticker_symbols of the companies “we made these decisions out of an abundance of caution amid unprecedented market_conditions and other_factors ” it is unclear to me at this point what some transactions means large transactions derivates on the stock buy orders in general if what is written in this article is true the technical_difficulties that some brokerages were stating yesterday evening are questionable maybe someone should read the terms of agreement of td their td_ameritrade account and comment if they are allowed to do this maybe some sec_rule aplies instead i am not familiar_with this yet a question for today when ma

In [19]:
len(results)
# WOW awesome, we got over 100k succesfull pulls.
# now lets go ahead and do some exploration.

291998

In [13]:
# lets save these results for further analysis
# first we are going to want to turn the lists into 

# extracted = pd.DataFrame({'sent':results['sent'].apply(lambda x: " ".join(x)), 'extracted':results['extracted'].apply(lambda x: " ".join(x))})

# extracted.to_csv('extracted.csv')

# Explore Results

In [20]:
c = collections.Counter(list(itertools.chain.from_iterable(results['extracted'])))
len(c)

801

In [21]:
counts = dict(c)

# lets see what the distribution of counts looks like.
fig = px.histogram(pd.DataFrame({'value':counts.values()}), x="value", nbins=20)
fig.show()
# yea so most of them are mentioned less than 1k times

In [22]:
counts

{'gme': 98908,
 'uber': 1091,
 'bb': 15287,
 'amc': 43504,
 'fb': 2175,
 'nok': 12998,
 'blackberry': 1312,
 'gamestop': 10585,
 'tsla': 11462,
 'tesla': 9004,
 'facebook': 1421,
 'tencent': 181,
 'chewy': 196,
 'game_stop': 644,
 'netflix': 1161,
 'spy': 6596,
 'microsoft': 905,
 'apple': 3227,
 'nokia': 3076,
 'baidu': 109,
 'palantir': 922,
 'pltr': 6203,
 'bbby': 877,
 'axdx': 43,
 'amazon': 2580,
 'jd': 708,
 'nflx': 1121,
 'pins': 170,
 'pypl': 270,
 'ibm': 212,
 'ge': 1217,
 'cisco': 91,
 'comcast': 99,
 'tiktok': 577,
 'irbt': 112,
 'prpl': 532,
 'amd': 7321,
 'nio': 3119,
 'snap': 2488,
 'bed_bath_beyond': 26,
 'spotify': 261,
 'spce': 2850,
 'tdoc': 71,
 'aal': 1128,
 'nakd': 3136,
 'ame': 53,
 'gnus': 548,
 'appl': 343,
 'sndl': 4550,
 'expr': 707,
 'plug': 871,
 'uavs': 125,
 'baba': 1546,
 'alibaba': 358,
 'intel': 764,
 'nspr': 34,
 'jagx': 223,
 'wtrh': 79,
 'koss': 580,
 'dpls': 31,
 'nkla': 1262,
 'idex': 349,
 'naked_brand': 87,
 'sundial_growers_sndl': 56,
 'fubo': 5

# Sentiment Analysis